# Many-body localization

This project uses dynamite to explore the phenomenon of many-body localization (MBL). Iterative methods (including the Krylov subspace methods upon which dynamite is built) have proved to be one of the only numerical methods that can effectively explore the MBL transition, in particular the region of parameter space in which the eigenvectors become thermal and thus tensor network methods break down.

We do not endeavor to write a full review of the physics of MBL in this README, but we refer interested readers to *TODO: add references*.

In this project we explore a model of nearest-neighbor Heisenberg interactions on a 1D chain, with random Z fields on each site:

$$H = \sum_{\left<i,j\right>} \vec{S}_i \cdot \vec{S}_j + \sum_i h_i S^z_i$$

where $\vec{S} = (S^x, S^y, S^z)$ and the subscripts indicate the index of the spin in the chain. The values of $h_i$ are drawn from a uniform distribution $\left[-h, h\right]$ where $h$ is a parameter that controls the strength of the disorder.

In the script `run_mbl.py` this is implemented by the `build_hamiltonian` function:

In [3]:
from dynamite import config
config.L = 10

from run_mbl import build_hamiltonian
build_hamiltonian(h=2)

<Operator on 10 spins: index_sum(0.25(σx[0]*σx[1] + σy[0]*σy[1] + σz[0]*σz[1]), sites 0 - 8) + -0.465σz[0] + 0.952σz[1] + -0.24σz[2] + -0.688σz[3] + -0.234σz[4] + 0.231σz[5] + -0.57σz[6] + 0.399σz[7] + 0.87σz[8] + -0.765σz[9]>

## Goals

In this project we plot two quantities that help us identify the MBL transition: the half-chain entanglement entropy of eigenstates, and an eigenvalue statistic called the *adjacent gap ratio*. The half chain entanglement entropy $S_{L/2}$ is simply the bipartite von Neumann entropy when half the spins are traced out. The MBL transition should correspond to a transition from volume law to area law entanglement. The adjacent gap ratio computes a measure of how likely eigenvalues are to be near each other. Let $\Delta_i$ be the gap between the $i^\mathrm{th}$ eigenvalue and the following one; that is, $\Delta_i = \lambda_{i+1} - \lambda_i$. Then, the adjacent gap ratio is defined as $r_i = \frac{\min \left( \Delta_i, \Delta_{i+1} \right)}{\max \left( \Delta_i, \Delta_{i+1} \right)}$. Random matrix theory tells us that in the thermal phase, the expectation value of $r$ should be $\left< r \right> \approx 0.53$, while in the localized phase $\left< r \right> \approx 0.39$.

The key feature that makes MBL so interesting is that the transition from volume to area law entanglement does not only occur in the ground state, but in excited states as well. This presents a great use case for dynamite's `target` eigenvalue solver, which finds the $k$ eigenvalues (and eigenvectors) closest to a target energy, where $k$ is user configurable. So, the plan is as follows: we will choose a few energies at various points in the spectrum, solve for some reasonable number (say, 32) eigenpairs near each of those points, and then compute the entanglement entropy and adjacent gap ratio for all of those eigenpairs.

## Usage